In [1]:
import os
import copy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  

X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.98, random_state=2)
XR, yR = make_regression(n_samples=100, n_features=2, noise=0, random_state=5)
X_train2, X_test2, y_train2, y_test2 = train_test_split(XR, yR, test_size=0.20, random_state=2)

def normalize_dataset(X):
    return tf.keras.utils.normalize(X)

c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
class LinearRegressor(tf.Module):
    def __init__(self, X, y, epochs=100, lr=0.001):
        self.X = X
        self.lr = lr
        self.y = y
        self.optimizer = tf.keras.optimizers.Adam(lr)
        self.mse = tf.keras.metrics.MeanSquaredError()
        self.accuracy = tf.keras.metrics.MeanSquaredLogarithmicError()
        self.precision = tf.keras.metrics.Precision()
    
    @tf.function
    def train_step(self):
        return tf.transpose(tf.multiply(tf.matmul(tf.linalg.inv(tf.matmul(tf.transpose(self.X), self.X)), tf.transpose(self.X)), self.y))

model = LinearRegressor(X_train2, y_train2, 1, 0.001)
print(model.train_step())


tf.Tensor(
[[ 7.63157356e-01 -3.33903513e-01]
 [ 3.68625643e-01 -4.78704138e-01]
 [ 4.63989054e+00  2.73544567e-01]
 [ 4.37465216e-01 -6.01878426e-03]
 [-5.64168816e-02  4.81583884e-01]
 [ 2.75573790e-02  3.46293520e-01]
 [ 3.13854988e-02 -5.52448581e-02]
 [ 6.38810304e-01  8.35957976e-02]
 [ 2.26540988e-01  9.75344934e-02]
 [-1.57187509e-02  3.81434312e-02]
 [ 2.30949782e-01 -2.57217869e-01]
 [ 1.22207423e+00  8.52512608e-02]
 [ 7.75335434e-01  7.04126915e-01]
 [ 2.71752685e-01 -3.33456044e-01]
 [ 1.18625596e-01  4.76278927e-02]
 [ 3.41435758e+00  1.40816846e+00]
 [ 5.10410288e+00 -4.30880482e-01]
 [-4.98817648e-02  2.39693001e-01]
 [ 2.08505616e+00  6.59597438e-01]
 [ 7.46805778e-02 -1.33370159e-01]
 [ 1.54747280e+00  9.65034575e-02]
 [ 1.89266328e+00  2.04019924e-01]
 [ 3.83494523e-01  5.06605095e-01]
 [ 5.87800272e-01  1.85970906e+00]
 [-1.51359085e-01  1.00360555e+00]
 [ 1.50619231e+00 -1.42749798e+00]
 [ 1.12030273e-01 -1.01318596e-01]
 [ 6.84250529e-02  1.68875889e-01]
 [ 6.2438